Running a large language models (LLM), namely llama2, locally on my Mac was the next logical step for me working through the [hacker's guide by Jeremy Howard](https://youtu.be/jkrNMKz9pWU?si=rh31HHhAnaKPZw1I). While it was possible to adjust Jeremy's approach on Hugging Face to also work on Apple Silicon, I focussed on [`llama.cpp`](https://github.com/ggerganov/llama.cpp) and its python binding [`llama-cpp-python`](https://github.com/abetlen/llama-cpp-python) to talk to llama2.

The whole journey consisted of the following steps, and I am going to take you though all of them to share my learnings along the way:

- Getting access to / downloading llama2
- [Running Llama2 via Hugging Face](https://github.com/chrwittm/lm-hackers/blob/main/20-local-llama-on-mac/10-running-llama2-on-mac1-hf.ipynb) and understanding why this is not a good approach on a Mac
- [Running Llama2 via `llama.cpp`/`llama-cpp-python`](https://github.com/chrwittm/lm-hackers/blob/main/20-local-llama-on-mac/20-running-llama2-on-mac2-cpp.ipynb) and understanding why this approach works a lot better on a Mac

<style>
  figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    text-align: center;
  }
</style>

<figure>
    <img src="llama-on-mac.png" alt="Dalle: A llama on a Mac" style="width:50%;">
    <figcaption>Dalle: A llama on a Mac</figcaption>
</figure>

## Getting Access to Llama2

First things first: Before, you can access the Llama2 model, you need to agree to Meta's the terms and conditions  for Llama2. As per the time of writing this, the process was as follows:

- Visit [the model's home page at Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-hf)
- Go to [Meta's website](https://ai.meta.com/resources/models-and-libraries/llama-downloads/), and complete the registration form
- Confirm the terms and conditions on the Hugging Face Website (see [screenshot](access-llama2-on-hf.png))

The approval only took a couple of minutes.

## Running Llama2 via Hugging Face

Trying to stick as closely as possible to the original [hacker's guide](https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb), I wanted to run LLama2 locally on my Mac using the Hugging Face API, just to see if it worked. Without Nvidia support, I needed to adapt the code to make it compatible with [Apple's Metal Framework](https://developer.apple.com/documentation/metalperformanceshaders). For all all the details, what needed to be done to run llama2 via the Hugging Face API, please [check out this notebook](https://github.com/chrwittm/lm-hackers/blob/main/20-local-llama-on-mac/10-running-llama2-on-mac1-hf.ipynb).

The final result was academically interesting, but performance left much to be desired 😉: What Jeremy's machine did in 2 seconds took my MacBook more than 3 minutes. There are probably a couple of reasons which produced this dramatic difference in performance:

- Nvidia memory throughput is a lot better then Apple's unified RAM
- The model I used was originally optimized and quantized for Nvidia GPUs. To run this model on my MacBook, I had to disable the 8-bit quantization (`load_in_8bit=False`) among other changes. While this adaptation was necessary for compatibility with Apple Silicon, it discarded all the optimizations.
- PyTorch's optimization for CUDA is probably still way better than its MPS optimization.

Here is a key learning: Running large language models (LLMs) locally requires more than brute force. Instead, hardware and software need to be aligned. Apple Silicon machines are extremely capable, but they need a different kind of optimization then Nvidia hardware. Consider the following analogy: Imagine you need to travel from Hamburg to Munich, and you have 2 hardware setups available, a car (let's say this represents Nvidia hardware) or a plane (let's say this represents Apple Silicon). Both these hardware setups require different optimizations to get from A to B.

Driving from Hamburg to Munich by car (representing Nvidia hardware), you optimize your path along the roads. If you used the plane instead (representing Apple Silicon), the same optimization would not work well. Attempting to navigate the plane on the roads, as you would a car, is highly impractical. Therefore, you would use a different way to optimize the path: You take public transport or a taxi to the airport, you fly from Hamburg to Munich, and again, you take public transport or a taxi to reach your final destination. On both hardware setups you have reached your Munich, but the underlying setup and optimizations differed significantly.

Therefore, let's hop on the plane, and let's explore a different way to run llama2 to on a Mac: Let's turn our attention to llama.cpp.

<style>
  figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    text-align: center;
  }
</style>

<figure>
    <img src="llama-driving-flying.png" alt="Dalle: A llama driving a car and another llama flying a plane on the road from Hamburg to Munich" style="width:75%;">
    <figcaption>Dalle: A llama driving a car and another llama flying a plane on the road from Hamburg to Munich</figcaption>
</figure>

## What is `llama.cpp`?

[`Llama.cpp`](https://github.com/ggerganov/llama.cpp) is an optimized library to run a large language model (LLM) like Llama2 on a Mac, but it also supports other platforms. How is this possible? For the details, please let me refer to this [tweet by Andrej Karpathy](https://twitter.com/karpathy/status/1691844860599492721) and for even more details to this [blog post by Finbarr Timbers](https://finbarr.ca/how-is-llama-cpp-possible/). Here are my takeaways:

- [`Llama.cpp`](https://github.com/ggerganov/llama.cpp) runs inference of LLMs in pure C/C++, therefore, it is significantly faster than implementations in higher languages like python.
- Additionally, [the mission](https://github.com/ggerganov/llama.cpp?tab=readme-ov-file#description) of the project  _"is to run the LLaMA model using 4-bit integer quantization on a MacBook"_. This means that numbers used to represent model weights and activations downsized from 32- or 16- bit floating points (the format of the base models) with 4-bit integers. This reduces memory usage and improves the performance and efficiency of the model during inference. The somewhat surprising thing is that model performance does not degrade by this downsizing.

When I mentioned before that I had to turn off quantization on Hugging Face, here we turn it on a again, just differently.

## How You Can Use llama.cpp from Python

The project [`llama-cpp-python`](https://github.com/abetlen/llama-cpp-python) serves as a binding for [`llama.cpp`](https://github.com/ggerganov/llama.cpp), providing access to the C++ API to Llama2 from Python.

In this context, a "[binding](https://en.wikipedia.org/wiki/Language_binding)" is a bridge that facilitates interaction between two programming languages, i.e. a layer of code that allows two programming languages to interact with each other. [`Llama.cpp`](https://github.com/ggerganov/llama.cpp) is written in C/C++, and the [`llama-cpp-python`](https://github.com/abetlen/llama-cpp-python) binding allows this C/C++ library to be utilized within a Python environment. Essentially, the Python code wraps around the C/C++ code so that it can be called from a Python environment.

While it might sound complicated, the concept is surprisingly accessible when you reduce the context to a simple example. To keep the focus in this blog post, I separated the exploration of C bindings into [this blog post]() (LINK).

## Installing `llama-cpp-python`

First, we need to [install](https://llama-cpp-python.readthedocs.io/en/latest/#installation) `llama-cpp-python` via `pip install llama-cpp-python`.

[Upgrading](https://llama-cpp-python.readthedocs.io/en/latest/#upgrading-and-reinstalling) is done via `pip install llama-cpp-python  --upgrade --force-reinstall --no-cache-dir`.

> 💡 Note: To execute the steps interactively, please check out [my related notebook](https://github.com/chrwittm/lm-hackers/blob/main/20-local-llama-on-mac/20-running-llama2-on-mac2-cpp.ipynb).

## Downloading the Model

For all my experiments, I used the following model: [TheBloke/Llama-2-7b-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF)

To download the model, please please run the code below, assuming that you have stored your Hugging Face access token in the `.env`-file. For additional insights/troubleshooting, please also check out [my previous blog post](https://chrwittm.github.io/posts/2024-01-27-how-to-call-openai-api/) / [my previous notebook](https://github.com/chrwittm/lm-hackers/blob/main/20-local-llama-on-mac/running-llm-on-mac1-hf.ipynb):

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv('HF_TOKEN')
os.environ['HF_TOKEN'] = token
!huggingface-cli login --token $HF_TOKEN
!wget -P ../models https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf

## Loading the Model

Loading the model, only required 2 lines of code. Let's talk about the parameters:

- `n_ctx=2048`: This sets the context window to 2048 tokens. The maximum number of tokens for this model is 4096.
- `verbose=False`: This makes the model less talkative. It only prints the actual results when prompted. Please feel free try turning it to `True` to get additional information from the model, not just the generated response.

In [ ]:
from llama_cpp import Llama
llm = Llama(model_path="../models/Llama-2-7b-chat/llama-2-7b-chat.Q4_K_M.gguf", n_ctx=2048, verbose=False)
#llm = Llama(model_path="../../../lm-hackers/models/llama-2-7b-chat.Q4_K_M.gguf", n_ctx=2048, verbose=False)

## Completion vs. Chat Completion Example

There are 2 ways we can talk to the LLM: The completion method literally does what it promises: It completes a prompt. For having a conversation with the LLM, we need to use Chat Completion.

As per the [Getting Started guide](https://llama-cpp-python.readthedocs.io/en/latest/#high-level-api), let's look at one example each on how to use the API:

Let's do text completion first.

In [8]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=128, stop=["Q:", "\n"], echo=True)
print(output['choices'][0]['text'])

Q: Name the planets in the solar system? A: 1. Mercury 2. Venus 3. Earth 4. Mars 5. Jupiter 6. Saturn 7. Uranus 8. Neptune


For the chat completion, let's re-write the code to reproduce the [example from the hackers guide](https://github.com/chrwittm/lm-hackers/blob/main/10-open-ai-api/accessing-openai-api.ipynb) to make the LLM talk about money in aussie slang.

In [9]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies whenever possible."

messages=[
    {"role": "system", "content": aussie_sys},
    {"role": "user", "content": "What is money?"}]

model_response = llm.create_chat_completion(messages = messages, stream=False)
print(model_response['choices'][0]['message']['content'])

  Fair dinkum, mate! Money, eh? It's like the oxygen we breathe, ya know? (laughs) Just kiddin', but seriously, money is like the lifeblood of society. It's what keeps the economy tickin' over and allows us to buy the things we need and want.
Think of it like this: money is like a big ol' pile of dough (get it? Dough? Like bread dough? Ah, never mind). We all gotta work hard to earn that dough, whether it's through our day job or by startin' our own business. And then we can use that dough to buy things like food, shelter, and a cold one at the pub after work.
But here's the thing: money ain't everything, mate. There's more to life than just makin' dough. We gotta find meaning and purpose in our lives, or else we'll be livin' like a dog (sorry, dogs!). So, while money's important, it's not the only thing that matters.
Now, I know some blokes might say, "Money, money, money! That's all that matters!" But let me tell you, mate, they're barkin' up the wrong tree (get it? Barkin' up the wr

## Conclusion

Using the right approach, running an LLM, in this case llama2, in a Jupyter notebook on a Mac is not really difficult. Once you sorted out the setup (like terms and conditions), starting up llama2 via `llama-cpp-python` only requires a few lines of code. Happy chatting!

<style>
  figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    text-align: center;
  }
</style>

<figure>
    <img src="aussie-llm-money.png" alt="Dalle: Aussie LLM depicted as kangaroo talking about money with Aussie slang and analogies" style="width:50%;">
    <figcaption>Dalle: Aussie LLM depicted as kangaroo talking about money with Aussie slang and analogies</figcaption>
</figure>